# Surface track from pinnacle ADCP
This script will produce a csv file with surface track derived from long range ADCP togehether with supporting figures.

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cmocean.cm as cmo
from cartopy.crs import SouthPolarStereo, PlateCarree
from pathlib import Path

from hugin import read_hugin_nav, add_AUV_nav
from pinnacle import find_surface_level
from plots import pcolormesh_offset

In [ ]:
cmap = cmo.tempo

## Load and combine pinnacle and auv data

In [ ]:
mission = 'NBP22_04'

if mission == 'ANA14B_01':
    ADCP_file = '../../data/ANA14B/ANA14B_01.nc'
    AUV_mission_folder = '../../data/ANA14B/ANA14B_01'
    
elif mission == 'NBP22_02':
    ADCP_file = '../../data/NBP22/NBP22_AUV2.nc'
    AUV_mission_folder = '../../data/NBP22/NBP2202_02'

elif mission == 'NBP22_03':
    ADCP_file = '../../data/NBP22/NBP22_AUV3.nc'
    AUV_mission_folder = '../../data/NBP22/NBP2202_03'
        
elif mission == 'NBP22_04':
    ADCP_file = '../../data/NBP22/NBP22_AUV4.nc'
    AUV_mission_folder = '../../data/NBP22/NBP2202_04'
    
save_folder = f"{mission}__surface_track/"

In [ ]:
ds = xr.open_dataset(ADCP_file)
ds

In [ ]:
nav = read_hugin_nav(AUV_mission_folder)
nav

In [ ]:
ds = add_AUV_nav(ds, nav, debug_plots=False)

In [ ]:
ds.SerEAAcnt.T.plot(cmap=cmap, vmax=140, figsize=(10,5))
plt.title('Raw data')
plt.show() 

## Remove bad data

In [ ]:
amp_threshold = 140
percentage_threshold = 0.25
pitch_threshold__100thDeg = 700
roll_threshold__100thDeg = 700
depth_threshold = 5

In [ ]:
ds.SerEAAcnt.plot.hist(bins=50);
plt.title('Amplitude')
plt.show()

ds.AnP100thDeg.plot.hist(bins=100)
plt.title('Pinnacle pitch')
plt.show()

ds.AnR100thDeg.plot.hist(bins=100)
plt.title('Pinnacle roll')
plt.show()

In [ ]:
above_threshold = ds.SerEAAcnt > amp_threshold
ping_ok = above_threshold.sum(dim='range') < ds.dims['range']/2

pitch_ok = abs(ds.AnP100thDeg) < pitch_threshold__100thDeg
roll_ok = abs(ds.AnR100thDeg) < roll_threshold__100thDeg

depth_ok = ds.NAV_DEPTH > depth_threshold

In [ ]:
ds.SerEAAcnt.where(ping_ok & pitch_ok & roll_ok & depth_ok).dropna('time').T.plot(cmap=cmap, vmax=140, figsize=(10,5))
plt.title('Cleaned data')
plt.show() 

## Time average

In [ ]:
time_factor = 30

variables = ['AnP100thDeg',
             'AnR100thDeg',
             'AnH100thDeg',
             'AnT100thDeg',
             'AnDepthmm',
             'SerEA1cnt',
             'SerEA2cnt',
             'SerEA3cnt',
             'SerEA4cnt',
             'SerEAAcnt',
             'NAV_DEPTH',
             'NAV_LATITUDE',
             'NAV_LONGITUDE',]
data = ds.where(ping_ok & pitch_ok & roll_ok & depth_ok).coarsen(time=time_factor, boundary='pad').mean()[variables]

In [ ]:
data.SerEAAcnt.dropna('time').T.plot(cmap=cmap, vmax=140, figsize=(10,5))
plt.title('Processed data')
plt.show() 

# Detect surface

In [ ]:
mode = 'max++'
zmax = 100
rmin = 200

data = find_surface_level(data, mode=mode, rmin=rmin, zmax=zmax)

# Export track

In [ ]:
Path(save_folder).mkdir(exist_ok=True)

variables = ['NAV_DEPTH', 'NAV_LONGITUDE', 'NAV_LATITUDE', 'surface_level', 'distance_to_surface'] 
df = data[variables].to_pandas().reset_index()
df.time = df.time.dt.round('1s') # Round to seconds

# Remove rows with nan
df = df[~np.isnan(df.surface_level)]

df.to_csv(f"{save_folder}surface_track.csv", index=False)

# Make figures

In [ ]:
plt.figure(figsize=(12,10))

ax = plt.axes(projection = SouthPolarStereo())
sc = ax.scatter(df['NAV_LONGITUDE'],df['NAV_LATITUDE'],c=df['surface_level'],
                transform = PlateCarree(), 
                s=20)
    
cbar = plt.colorbar(sc, label='surface level (m)', shrink=0.5)

# Adjust grid lines
gl = ax.gridlines(draw_labels = True, 
                  #y_inline = False, # force y-axis ticks to be outside the plot
                 )
gl.bottom_labels = True
gl.top_labels    = True
gl.left_labels   = True                  
gl.right_labels  = True

plt.title("Surface level detections")
plt.savefig(f"{save_folder}surface_track_map")
plt.show()

In [ ]:
# Check for time lag:
(data.AnDepthmm/1000).plot(label='pinnacle')
data.NAV_DEPTH.plot(label='auv',linestyle='dashed')
plt.legend()
plt.ylabel('depth (m)')
plt.gca().invert_yaxis()
plt.title('Time lag')

plt.savefig(f"{save_folder}time_lag")

In [ ]:
plt.figure(figsize=(12,5))
pcolormesh_offset(data.time.values, -data.range.values, data.SerEAAcnt.T.values, data.AnDepthmm.values/1000,
                  cmap=cmap, vmax=140)
plt.gca().invert_yaxis()
plt.ylabel('depth (m)')
plt.title('Backscatter amplitude')

plt.savefig(f"{save_folder}backscatter")
plt.show()

In [ ]:
plt.figure(figsize=(12,5))
pcolormesh_offset(data.time.values, -data.range.values, data.SerEAAcnt.T.values, data.AnDepthmm.values/1000,
                  cmap=cmap, vmax=140)
plt.gca().invert_yaxis()
plt.ylabel('depth (m)')

plt.plot(data.time.values, -data.surface_level.values, '.', color='magenta')
plt.title('Surface detections')
plt.savefig(f"{save_folder}surface_detections")
plt.show()

In [ ]:
df.surface_level.plot.hist(bins=100)
plt.ylabel('count')
plt.xlabel('surface level (m)')
plt.grid()
plt.title('Surface level histogram')
plt.savefig(f"{save_folder}surface_level_histogram")

In [ ]:
parameter_file = f"{save_folder}parameters.txt"
Path(parameter_file).touch()
f = open(f"{save_folder}parameters.txt", 'w')
f.writelines(f"ampltiude threshold: {amp_threshold}\n" )
f.writelines(f"percentage threshold: {percentage_threshold}\n")
f.writelines(f"min depth: {depth_threshold}\n")
f.writelines(f"max pinnacle pitch (100th deg): {pitch_threshold__100thDeg}\n")
f.writelines(f"max pinnacle roll (100th deg): {roll_threshold__100thDeg}\n")
f.writelines(f"surface detection method: {mode}\n")
f.writelines(f"zmax: {zmax}\n")
f.writelines(f"rmin: {rmin}\n")
f.close()